In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import astra
astra.set_gpu_index((0, 1, 2, 3))

from AdversarialRegularizer import AdversarialRegulariser
from networks import AlexNet_3D
from networks_FoV import ResNetL2_local
from ellipsgen.generate_data_function import get_batch
import numpy as np
import ellipsgen.CCB_class as CT
import ellipsgen.phantom_class as ph
import matplotlib.pyplot as plt
import time

BATCH_SIZE = 4
LOOPS = 2
IMG_SIZE = 128
STEPS = 100

In [2]:
# Parameter choices. Heuristic in the BWGAN paper: Choose GAMMA as average dual norm of clean image
# LMB should be bigger than product of norm times dual norm.

# For s=0.0, this implies GAMMA =1.0
# For s=1.0, have GAMMA = 10.0 as realisitc value
S = 0.0
LMB = 10.0
GAMMA = 1.0
CUTOFF = 20.0

saves_path = '/export/scratch1/home/voxels-gpu0/codesprint_learned_prior/AdvRegSaves/test2/ell_noisy_' + str(IMG_SIZE)
regularizer = AdversarialRegulariser(saves_path, IMAGE_SIZE=(None, IMG_SIZE, IMG_SIZE, IMG_SIZE, 1), NETWORK=AlexNet_3D,
                                     s=S, cutoff=CUTOFF, lmb=LMB, gamma=GAMMA)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use keras.layers.conv3d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling3d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /export/scratch1/home/voxels-gpu0/codesprint_learned_prior/AdvRegSaves/test2/ell_noisy_128/Data/model-400
Save restored


In [3]:
def load_pair(i):
    prefix = '../noisy_' + str(IMG_SIZE) + '/'
    fdk = np.load(prefix + 'FDK/dataset' + str(i) + '.npy')
    gt = np.load(prefix + 'GT/dataset' + str(i) + '.npy')
    
    return fdk, gt


def generate_pairs(amount=1):
    num_angles = 360
    noise = ['Poisson', 2 ** 14] # Do not go below 2 ** 8, lower number in the 2nd argument means more noise
    num_vox = IMG_SIZE
    
    voxels = [num_vox, num_vox, num_vox]
    src_rad = 10
    det_rad = 0

    fdks = np.empty((amount, *voxels))
    gts = np.empty_like(fdks)

    for i in range(amount):
        data_obj = ph.phantom(voxels, '22 Ellipses', num_angles, noise, src_rad, det_rad)
        gts[i, ...] =  data_obj.f

        case = CT.CCB_CT(data_obj)
        fdks[i, ...] = case.do_FDK()
        
        if i == 0:
            plt.figure()
            plt.imshow(gts[i, int(IMG_SIZE/2), ...])
            plt.show()
            
            plt.figure()
            plt.imshow(fdks[i, int(IMG_SIZE/2), ...])
            plt.show()
            
    
    return (gts, fdks)
    

def evaluate(gt, adv):
    # gt, adv = get_batch(eval_data=True, noise_levels=['01', '016'], methods=['EM', 'SGD'])
    regularizer.test(groundTruth=gt, adversarial=adv)

    
def train(steps):
    for k in range(steps):        
#         if k%50 == 0:
#             gt, adv = generate_pairs(amount=1) # batch, x, y, z, channel
#             evaluate(gt, adv)
#         else:
            gts, advs = load_pair(k) # batch, x, y, z, channel
    
            print(k)
            t = time.time()
            regularizer.train(groundTruth=gts, adversarial=advs, learning_rate=LEARNING_RATE)
            print(time.time() - t)
            
    regularizer.save()

In [4]:
LEARNING_RATE = 0.00005
for k in range(LOOPS):
    train(STEPS)

LEARNING_RATE = 0.00002
for k in range(LOOPS):
    train(STEPS)

0
39.056464433670044
1
0.7974565029144287
2
0.7970256805419922
3
0.7937674522399902
4
0.7926039695739746
5
0.794792652130127
6
0.7958114147186279
7
0.7968182563781738
8
0.7946932315826416
9
0.7937488555908203
10
0.7867951393127441
11
0.7902712821960449
12
0.790992021560669
13
0.7874505519866943
14
0.7898509502410889
15
0.7942934036254883
16
0.7947471141815186
17
0.7947196960449219
18
0.7939338684082031
19
0.7900619506835938
20
0.7887344360351562
21
0.7955055236816406
22
0.7953243255615234
23
0.7939460277557373
24
0.7943220138549805
25
0.7938730716705322
26
0.7938699722290039
27
0.7933175563812256
28
0.7958948612213135
29
0.7962312698364258
30
0.7943720817565918
31
0.7946376800537109
32
0.7933287620544434
33
0.7936182022094727
34
0.79636549949646
35
0.7960026264190674
36
0.796921968460083
37
0.7964968681335449
38
0.7938807010650635
39
0.7969632148742676
40
0.7935624122619629
41
0.7931480407714844
42
0.7965080738067627
43
0.7958014011383057
44
0.7966444492340088
45
0.7976493835449219
46
